In [36]:
# 模块测试 梯度测试
from turtle import forward
import torch 
import torch.nn as nn

vgg = nn.Sequential(
    nn.Conv2d(3, 3, (1, 1)),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(3, 64, (3, 3)),
    nn.ReLU(),  # relu1-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),  # relu1-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 128, (3, 3)),
    nn.ReLU(),  # relu2-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),  # relu2-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 256, (3, 3)),
    nn.ReLU(),  # relu3-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 512, (3, 3)),
    nn.ReLU(),  # relu4-1, this is the last layer used
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU()  # relu5-4
)

def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    # assert (len(size) == 4)
    other = size[:-2]
    feat_var = feat.reshape(*other, -1).var(dim=-1) + eps
    feat_std = feat_var.sqrt().reshape(*other, 1, 1)
    feat_mean = feat.reshape(*other, -1).mean(dim=-1).reshape(*other, 1, 1)
    return feat_mean, feat_std


def adaIN(content_feat, style_feat):
    assert (content_feat.size()[:-2] == style_feat.size()[:-2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    normalized_feat = (content_feat - content_mean) / content_std
    return normalized_feat * style_std + style_mean

def oldblock_adaIN(content_feat, style_feat, block = 16):
    assert (content_feat.size()[:-2] == style_feat.size()[:-2])
    assert content_feat.size()[-1] % block == 0
    assert content_feat.size()[-2] % block == 0
    size = content_feat.size()
    N, C, H, W = size
    newC = C * size[-1] * size[-2] / block / block
    return adaIN(content_feat.reshape(N,C,H // block, block, W // block, block).transpose(3,4),
                                        style_feat.reshape(N,C,H // block, block, W // block, block).transpose(3,4)
                                        ).transpose(3,4).reshape(size)

def block_adaIN(content_feat, style_feat, block = 16):
    assert (content_feat.size()[:-2] == style_feat.size()[:-2])
    content_feat = blockzation(content_feat, block)
    style_feat = blockzation(style_feat, block)
    return  unblockzation(adaIN(content_feat, style_feat))

def blockzation(feat, block = 16):
    H, W = feat.size()[-2:]
    assert H % block == 0
    assert W % block == 0
    size = feat.size()[:-2]
    feat = feat.reshape(*size, H // block, block, W // block, block).transpose(-2, -3)
    return feat

def unblockzation(feat):
    size = feat.size()
    H = size[-4] * size[-2]
    W = size[-3] * size[-1]
    size = size[:-4]
    return feat.transpose(-2, -3).reshape(*size, H, W)

class Net(nn.Module):
    def __init__(self, encoder, level = 1):
        super(Net, self).__init__()
        enc_layers = list(encoder.children())[:44]
        self.enc_1 = nn.Sequential(*enc_layers[:4])  # input -> relu1_1
        self.enc_2 = nn.Sequential(*enc_layers[4:11])  # relu1_1 -> relu2_1
        self.enc_3 = nn.Sequential(*enc_layers[11:18])  # relu2_1 -> relu3_1
        self.enc_4 = nn.Sequential(*enc_layers[18:31])  # relu3_1 -> relu4_1
        self.enc_5 = nn.Sequential(*enc_layers[31:44])  # relu4_1 -> relu5_1

        self.level = level
        # fix the encoder
        for name in ['enc_1', 'enc_2', 'enc_3', 'enc_4', 'enc_5']:
            for param in getattr(self, name).parameters():
                param.requires_grad = False

    def encode_with_intermediate(self, input):
        results = [input]
        for i in range(4):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            results.append(func(results[-1]))
        return results[1:]
    
    def encode_with_level(self, input, level):
        for i in range(level):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            input = func(input)
        return input

    def forward(self, x):
        return self.encode_with_level(x, self.level)


In [10]:
import torch
def dynamic_adj_add(vec1, vec2):
    assert vec1.shape == vec2.shape
    shape = vec1.shape
    vec1 = vec1.view(shape[0], -1)
    vec2 = vec2.view(shape[0], -1)
    v1v1 = (vec1 * vec1).mean(dim = 1)
    v1v2 = (vec1 * vec2).mean(dim = 1)
    v2v2 = (vec2 * vec2).mean(dim = 1)
    gamma = min_norm_element_from2(v1v1, v1v2, v2v2).view(shape[0], 1)
    return (gamma  * vec1 + (1 - gamma) * vec2).view(shape)

def min_norm_element_from2(v1v1, v1v2, v2v2):
        divide = v1v1+v2v2 - 2*v1v2
        gamma = -1.0 * ( (v1v2 - v2v2) / (v1v1+v2v2 - 2*v1v2))
        gamma = torch.where(torch.isnan(gamma), torch.full_like(gamma, 1), gamma)
        return gamma.clamp(0, 1)

In [11]:
x = torch.ones(64,2)
y = torch.ones(64,2)
y[:,0] = .05

print(dynamic_adj_add(x,x))

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1

In [45]:
# 待测试方案
# 以下每行的排列组合
##  是否使用blockadain初始化
##  MSE or KL 
##  仅仅调节std和mean (blockadain level shape (batch, channel, area, area, 1, 1))还是直接调节feature map
###  MSE with std&mean 是直接std和mean分别的MSE
##  迭代次数 (考虑是用adam之类的优化器)
##  迭代的学习率
##  差分值的系数 
##  vgg网络深度

# 一个 optimization
# 此外还需要实现加噪信号的block 均值和方差的计算(推一推公式)
# basic
# x_t = a * x + b * z
# E(x_t) = a * E(x)
# Var(x_t) = a^2 * var(x) + b^2 
# 计算好ref image的block level的均值和方差之后, 可以通过计算的方式直接得到需要的均值和方差

####### things to adjust

### 梯度方案很容易偏色
### 仅仅迁移均值方差细节迁移不到位
### 两种方案合在一起, 先做neural feature的细节transfer, 再做image level的blockAdain
init_with_blockadain = True
# MSE is the MSE with adained feature map, While KL is the KL divergence of two distribulation, 
# but it has the same best optimization point with L2 distance between means, and between vars, further try to use multi task opt methods
losstype = "KL" # "MSE" or "KL"
# when use affine, the image level features only changes as affine
affine = False # keep image features affine
iterate_times = 30000 # the iterate times
iterate_lr = 1e-1 / 64 # the iterate learning rate 
difference_std = 7 # the std of difference / the std of the score
vggdeep = 1 # the feature extractor depth 1/2/3
area = 16 # in 1d how many part to divide 8/16/32

vgg.load_state_dict(torch.load("/home/sunsk/Models/vgg/vgg_normalised.pth"))

net = Net(vgg, level = vggdeep)
net.eval()
x = torch.randn(64,3,256,256) # source 
y = torch.randn(64,3,256,256)*3 + 5 # target features
H, W = x.shape[-2:]
blocked_x = blockzation(x, block = H // area)
blocked_y = blockzation(y, block= H // area)
# prepare features to optimization
with torch.enable_grad():
    if affine:
        blocked_x_mean, blocked_x_std = calc_mean_std(blocked_x) # [batchsize, channel, area, area, 1, 1]
        blocked_y_mean, blocked_y_std = calc_mean_std(blocked_y) # can be caltulated before
        if init_with_blockadain:
            mean = blocked_y_mean.detach().requires_grad_(True) # need to be updated to block level
            std = blocked_y_std.detach().requires_grad_(True)
        else:
            mean = blocked_x_mean.detach().requires_grad_(True)
            std = blocked_x_std.detach().requires_grad_(True)
        
        normalized_blocked_x = (blocked_x - blocked_x_mean) / blocked_x_std
        blocked_x_in = normalized_blocked_x * std + mean
    else:
        if init_with_blockadain:
            blocked_x_in = adaIN(blocked_x, blocked_y).detach().requires_grad_(True)
        else:
            blocked_x_in = blocked_x.detach().requires_grad_(True)

    x_feat = net(unblockzation(blocked_x_in))
    y_feat = net(y)
    H, W = x_feat.shape[-2:]
    blocked_x_feat = blockzation(x_feat,block = H // area)
    blocked_y_feat = blockzation(y_feat,block = H // area)

    blocked_y_feat_mean, blocked_y_feat_std = calc_mean_std(blocked_y_feat)

    for _ in range(iterate_times):
        print(blocked_x_in.mean())
        print((blocked_x_in - blocked_y).norm())
        if losstype == "MSE":
            blocked_target = adaIN(blocked_x_feat, blocked_y_feat)
            loss = (blocked_x_feat - blocked_target) ** 2
        # elif losstype == "KL":
        else:
            blocked_x_feat_mean, blocked_x_feat_std = calc_mean_std(blocked_x_feat)
            # original but not stable loss
            # loss = x_var / y_var + (x_mean - y_mean) / y_var - torch.log(x_var / y_var) - 1
            # stable loss
            loss = (blocked_x_feat_mean - blocked_y_feat_mean) ** 2 + (blocked_x_feat_std - blocked_y_feat_std) ** 2

        if affine:
            grad1 = torch.autograd.grad(loss.sum(), std, retain_graph= True)[0]
            grad2 = torch.autograd.grad(loss.sum(), mean)[0]
            std = std - iterate_lr * grad1
            mean = mean - iterate_lr * grad2
            blocked_x_in = normalized_blocked_x * std + mean
        else:
            grad = torch.autograd.grad(loss.sum(), blocked_x_in)[0]
            blocked_x_in = blocked_x_in - iterate_lr * grad
        x_feat = net(unblockzation(blocked_x_in))
        blocked_x_feat = blockzation(x_feat,block = H // area)
        print((blocked_x_feat - blocked_y_feat).norm())
        # target = block_adaIN(x_feat, y_feat)

tensor(-9.1397e-05, grad_fn=<MeanBackward0>)
tensor(20983.5996, grad_fn=<CopyBackwards>)
tensor(528136.9375, grad_fn=<CopyBackwards>)
tensor(0.0032, grad_fn=<MeanBackward0>)
tensor(21011.6348, grad_fn=<CopyBackwards>)
tensor(532531.9375, grad_fn=<CopyBackwards>)
tensor(0.0068, grad_fn=<MeanBackward0>)
tensor(21041.2070, grad_fn=<CopyBackwards>)
tensor(538549.3750, grad_fn=<CopyBackwards>)
tensor(0.0105, grad_fn=<MeanBackward0>)
tensor(21070.6367, grad_fn=<CopyBackwards>)
tensor(545544.9375, grad_fn=<CopyBackwards>)
tensor(0.0144, grad_fn=<MeanBackward0>)
tensor(21098.6582, grad_fn=<CopyBackwards>)
tensor(552861.8125, grad_fn=<CopyBackwards>)
tensor(0.0185, grad_fn=<MeanBackward0>)
tensor(21124.4336, grad_fn=<CopyBackwards>)
tensor(560169.1250, grad_fn=<CopyBackwards>)
tensor(0.0228, grad_fn=<MeanBackward0>)
tensor(21147.4727, grad_fn=<CopyBackwards>)
tensor(567303.5000, grad_fn=<CopyBackwards>)
tensor(0.0271, grad_fn=<MeanBackward0>)
tensor(21167.4727, grad_fn=<CopyBackwards>)
tensor(5

KeyboardInterrupt: 

In [76]:
x = torch.rand(2, 3, 64, 64)
y = torch.ones(2, 3, 64, 64)

In [77]:
print(block_adaIN(x, y).shape)

torch.Size([2, 3, 64, 64])


In [90]:
### grad 测试

import torch
import numpy as np
from torchvision import models

func = models.resnet101(pretrained=True)
data = torch.rand(64,3,64,64)
func.eval()
print(func(data).shape)

torch.Size([64, 1000])


In [87]:
with torch.enable_grad():
    x_in = data.detach().requires_grad_(True)
    logits = func(x_in).view(64,-1).sum(dim=-1)
    print(torch.autograd.grad(logits.sum(), x_in)[0].norm())
    print(64*12.6733)

tensor(811.0899)
811.0912


In [33]:
import torch as th
def dynamic_adj_add(vec1, vec2):
    # print(vec1.shape)
    assert vec1.shape == vec2.shape
    shape = vec1.shape
    vec1 = vec1.view(shape[0], -1)
    vec2 = vec2.view(shape[0], -1)
    v1v1 = (vec1 * vec1).mean(dim = 1)
    v1v2 = (vec1 * vec2).mean(dim = 1)
    v2v2 = (vec2 * vec2).mean(dim = 1)
    gamma = min_norm_element_from2(v1v1, v1v2, v2v2).view(shape[0], 1)
    coef = ((1 - gamma)/gamma).clamp(0,30)
    coef = th.where(th.isnan(coef), th.full_like(coef, 0), coef)
    # return (gamma * vec1 + (1 - gamma) * vec2).view(shape)
    return (vec1 + coef * vec2).view(shape)

def min_norm_element_from2(v1v1, v1v2, v2v2):
    divide = v1v1+v2v2 - 2*v1v2
    gamma = (v2v2 - v1v2) / divide
    gamma = th.where(th.isnan(gamma), th.full_like(gamma, 0), gamma)
    return gamma.clamp(0, 1)

# veclist shape: [batch, number, others]
# 方案一, 提取跟diffuison 方向垂直的分量, 将这些分量想办法dynamic化
# 方案二, 将当前的方案暴力拓展到三个变量的情况
def frank_wolfe_solver(veclist, ep = 1e-4, maxnum = 20):
    shape = veclist.shape
    veclist = veclist.view(shape[0], shape[1], -1) # shape [B, N, O]
    M = veclist @ veclist.transpose(1,2) # shape [B, N, N]
    a = (th.ones(shape[:2]) / shape[1]).unsqueeze(1).to(veclist.device) # shape [B, 1, N]
    for _ in range(maxnum):
        minrank = th.argmin(a @ M, dim = 2) # shape [B, 1]
        minonehot = th.zeros(shape[:2]).to(veclist.device).scatter_(1, minrank, 1).unsqueeze(1) # shape [B, 1, N]
        gamma = min_norm_element_from2(minonehot @ M @ minonehot.transpose(1,2),minonehot @ M @ a.transpose(1,2), a @ M @ a.transpose(1,2)).reshape(-1, 1, 1)
        # minvec = th.diagonal(veclist[:,minrank]).transpose(0,1)
        a = (1-gamma)* a + gamma * minonehot
        if th.abs(gamma).mean()< ep:
            return a
    return a

veclist = th.rand(3,3,3,64,64).to(device="cuda")
a = frank_wolfe_solver(veclist)
print(a)


In [26]:
import torch
import torchvision.models as models

path = "/home/sunsk/Models/resnet50/resnet50-19c8e357.pth"
resnet50 = models.resnet50(pretrained=False)
resnet50.load_state_dict(torch.load(path))
resnet50.eval()
print(len(list(resnet50.children())))
newmodel = torch.nn.Sequential(*(list(resnet50.children())[:6]))
cos = torch.nn.CosineSimilarity(eps = 1e-6)
batch = x.shape[0]
x = torch.rand(64,3,256,256)
y = torch.rand(64,3,256,256)
xh = newmodel(x)
yh = newmodel(y)
cossimi = cos(xh, yh).mean() * batch
print(xh.shape)
print(cossimi)


10
torch.Size([64, 512, 32, 32])
tensor(48.5318, grad_fn=<MulBackward0>)


In [14]:
import torch
x = torch.rand(1, 1, 3, 1)
y = torch.rand(1, 1, 3, 1)
y[:,:,:2,:] = 0
print(x)
print(y)
print(get_vertical_component(x, y, independdims=2))

tensor([[[[0.7101],
          [0.0259],
          [0.5566]]]])
tensor([[[[0.0000],
          [0.0000],
          [0.9167]]]])
torch.Size([1, 1, 3, 1])
1 1 3 1
tensor([[[[0.7101],
          [0.0259],
          [0.0000]]]])
